In [ ]:
import pandas as pd
import requests
import numpy as np
import json
import re
from gvol import GVol
from plotly.subplots import make_subplots
from datetime import datetime

GVOL_API_KEY = "GVOL_API_KEY"
gvol_client = GVol(header='gvol-lite',gvol_api_key=GVOL_API_KEY)
url = "https://app.pinkswantrading.com/graphql"
pd.set_option('display.max_columns', None)

In [ ]:
# Prepare the payload for the data request
# This is a GraphQL query that fetches live fixed delta surface data from the Deribit exchange for BTC
payload="{\"query\":\"\\tquery LiveDeribitFixedDeltaSurface(\\n\\t\\t$exchange: ExchangeEnumType, \\n        $symbol: BTCOrETHEnumType\\n\\t) {\\n\\t\\t LiveDeribitFixedDeltaSurface(exchange:$exchange, symbol:$symbol) {\\n             date\\n            \\n    currency\\n    spot\\n    atm7\\n    ThirtyFiveDelta7Put\\n    ThirtyFiveDelta7Call\\n    TwentyFiveDelta7Put\\n    TwentyFiveDelta7Call\\n    FifteenDelta7Put\\n    FifteenDelta7Call\\n    FiveDelta7Put\\n    FiveDelta7Call\\n    atm30\\n    ThirtyFiveDelta30Put\\n    ThirtyFiveDelta30Call\\n    TwentyFiveDelta30Put\\n    TwentyFiveDelta30Call\\n    FifteenDelta30Put\\n    FifteenDelta30Call\\n    FiveDelta30Put\\n    FiveDelta30Call\\n    atm60\\n    ThirtyFiveDelta60Put\\n    ThirtyFiveDelta60Call\\n    TwentyFiveDelta60Put\\n    TwentyFiveDelta60Call\\n    FifteenDelta60Put\\n    FifteenDelta60Call\\n    FiveDelta60Put\\n    FiveDelta60Call\\n    atm90\\n    ThirtyFiveDelta90Put\\n    ThirtyFiveDelta90Call\\n    TwentyFiveDelta90Put\\n    TwentyFiveDelta90Call\\n    FifteenDelta90Put\\n    FifteenDelta90Call\\n    FiveDelta90Put\\n    FiveDelta90Call\\n    atm180\\n    ThirtyFiveDelta180Put\\n    ThirtyFiveDelta180Call\\n    TwentyFiveDelta180Put\\n    TwentyFiveDelta180Call\\n    FifteenDelta180Put\\n    FifteenDelta180Call\\n    FiveDelta180Put\\n    FiveDelta180Call\\n\\t\\t}\\n\\t}\",\"variables\":{\"exchange\":\"deribit\",\"symbol\":\"BTC\"}}"

# Set up the request headers
# The x-oracle header is used for authentication
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

# Send the GET request to the specified URL
# The response will contain the requested data
response = requests.request("GET", url, headers=headers, data=payload)

# Parse the response text into JSON format
data = json.loads(response.text)

# Convert the JSON data into a pandas DataFrame
# The json_normalize function is used to flatten the JSON into a table
df = pd.json_normalize(data['data']['LiveDeribitFixedDeltaSurface'])

The following cell is used to fetch and prepare hourly fixed delta surface data for a given symbol and date range. The data is fetched by sending a GET request to a specified URL. The request includes a payload which is a GraphQL query specifying the data to fetch, as well as headers for authentication and setting the content type.

The response from the server is in JSON format, which is parsed and converted into a pandas DataFrame for easier manipulation and analysis. The 'ts' column, which represents timestamps, is converted to datetime format and set as the index of the DataFrame. This makes it easier to work with the data as a time series.

This cell is an essential first step in any analysis involving this data, as it provides the raw data that will be used in subsequent steps.

In [ ]:
# Get yesterday's date
yesterday = datetime.today().strftime('%Y-%m-%d')

# Prepare the GraphQL query
payload = f"""{{
  "query": "\\tquery HourlyFixedDeltaSurface(\\n$symbol: SymbolEnumType,\\n$dateStart: String,\\n$dateEnd: String) {{\\nHourlyFixedDeltaSurface: HourlyFixedDeltaSurface(\\nsymbol: $symbol, \\ndateStart: $dateStart,\\ndateEnd: $dateEnd) {{\\nts\\ncurrency\\nindexPrice\\natm7\\natm30\\natm60\\natm90\\natm180\\nThirtyFiveDelta7Put\\nThirtyFiveDelta7Call\\nTwentyFiveDelta7Put\\nTwentyFiveDelta7Call\\nFifteenDelta7Put\\nFifteenDelta7Call\\nFiveDelta7Put\\nFiveDelta7Call\\nThirtyFiveDelta30Put\\nThirtyFiveDelta30Call\\nTwentyFiveDelta30Put\\nTwentyFiveDelta30Call\\nFifteenDelta30Put\\nFifteenDelta30Call\\nFiveDelta30Put\\nFiveDelta30Call\\nThirtyFiveDelta60Put\\nThirtyFiveDelta60Call\\nTwentyFiveDelta60Put\\nTwentyFiveDelta60Call\\nFifteenDelta60Put\\nFifteenDelta60Call\\nFiveDelta60Put\\nFiveDelta60Call\\nThirtyFiveDelta90Put\\nThirtyFiveDelta90Call\\nTwentyFiveDelta90Put\\nTwentyFiveDelta90Call\\nFifteenDelta90Put\\nFifteenDelta90Call\\nFiveDelta90Put\\nFiveDelta90Call\\nThirtyFiveDelta180Put\\nThirtyFiveDelta180Call\\nTwentyFiveDelta180Put\\nTwentyFiveDelta180Call\\nFifteenDelta180Put\\nFifteenDelta180Call\\nFiveDelta180Put\\nFiveDelta180Call\\n}}\\n}}",
  "variables": {{
    "symbol": "ETH",
    "dateStart": "2023-05-01",
    "dateEnd": "{yesterday}"
  }}
}}"""

# Define headers for the request
headers = {
  'x-oracle': GVOL_API_KEY,
  'Content-Type': 'application/json',
  'accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.9'
}

# Send GET request and parse response
response = requests.request("GET", url, headers=headers, data=payload)
data = json.loads(response.text)

# Flatten JSON data into a DataFrame
df_hist = pd.json_normalize(data['data']['HourlyFixedDeltaSurface'])

# Convert 'ts' column to datetime and set as index
df_hist['ts'] = pd.to_datetime(df_hist['ts'], unit='ms')
df_hist.set_index('ts', inplace=True)

The following cell is used to preprocess the data and extract relevant features for further analysis. The DataFrame is first "melted" to a long-format, making it easier to manipulate and visualize.

Features such as 'delta', 'expiration', and 'option_type' are extracted from the 'variable' column using regular expressions. The 'delta' values are mapped from strings to numeric values for easier processing.

Afterwards, any rows with missing values in the new columns are dropped, and the 'expiration' values are converted to numeric. Finally, the DataFrame is grouped by 'delta' and 'expiration', and the mean 'volatility' for each group is calculated.

This processed data can now be used for further analysis and visualization, providing insights into the relationship between 'delta', 'expiration', and 'volatility'.

In [ ]:
# Melt the DataFrame to make it long-format
# This makes it easier to manipulate and visualize
df_melt = df.melt(id_vars=['date', 'currency', 'spot'], var_name='variable', value_name='volatility')

# Create a mapping from string deltas to integers
# This allows us to convert the string deltas in the 'variable' column to numeric values
delta_mapping = {
    'Five': 5,
    'Fifteen': 15,
    'TwentyFive': 25,
    'ThirtyFive': 35
}

# Use regular expressions to extract the delta, expiration, and option type from the 'variable' column
# The 'delta' is the part before 'Delta', the 'expiration' is the part after 'Delta' and before 'Put' or 'Call', and the 'option_type' is 'Put' or 'Call'
df_melt['delta'] = df_melt['variable'].apply(lambda x: re.findall(r'(\w+)Delta', x)[0] if re.findall(r'(\w+)Delta', x) else None)
df_melt['delta'] = df_melt['delta'].apply(lambda x: delta_mapping.get(x, np.nan))
df_melt['expiration'] = df_melt['variable'].apply(lambda x: re.findall(r'Delta(\d+)', x)[0] if re.findall(r'Delta(\d+)', x) else None)
df_melt['option_type'] = df_melt['variable'].apply(lambda x: 'Call' if 'Call' in x else 'Put')

# Remove rows where any of the new columns are None
# Convert 'expiration' to numeric values for further processing
df_melt = df_melt.dropna()
df_melt['expiration'] = pd.to_numeric(df_melt['expiration'])

# Group by 'delta' and 'expiration', and calculate the mean of 'volatility'
# This gives us the average volatility for each combination of 'delta' and 'expiration'
df_melt = df_melt.groupby(['delta', 'expiration'], as_index=False).mean()

### Plot Functions

These plots provide various perspectives on the dynamics of options implied volatilities. They show time series of raw and normalized implied volatilities for different delta options types (calls and puts). Some plots display the risk reversal, which measures the difference in implied volatility between similar out-of-the-money call and put options, giving insight into market sentiment. Others show implied volatilities normalized by at-the-money (ATM) implied volatility to highlight relative changes. These visualizations are useful tools for analyzing market behavior and trends, facilitating options trading strategies, and conducting comparative analysis across different time periods, options types, and expiration levels. Each plot has been constructed with user-friendly titles for easy interpretation.

In [ ]:
def delta_plot(df, option_type, delta):
    fig = go.Figure()

    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"]) and option_type in column]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column and option_type in column]

    for column in filtered_cols:
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[column],
            mode='lines',
            name=column
        ))

    fig.update_layout(
        title=f"Time Series of Implied Volatility for {delta} Delta {option_type} Options",
        yaxis_title='Implied Volatility',
        template='plotly_dark'
    )

    fig.show()

def delta_rr_plot(df, delta, diff_order):
    fig = go.Figure()
    
    df_diff = pd.DataFrame(index=df.index)
    
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"])]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column]

    for column in filtered_cols:
        if 'Put' in column and diff_order == 'P-C':
            corresponding_call_column = column.replace('Put', 'Call')
            if corresponding_call_column in df.columns:
                difference_col_name = column.replace('Put', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_call_column]
        elif 'Call' in column and diff_order == 'C-P':
            corresponding_put_column = column.replace('Call', 'Put')
            if corresponding_put_column in df.columns:
                difference_col_name = column.replace('Call', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_put_column]
    
    for column in df_diff.columns:
        fig.add_trace(go.Scatter(
            x=df_diff.index,
            y=df_diff[column],
            mode='lines',
            name=column
        ))

    fig.update_layout(
        title=f"Time Series of Implied Volatility Risk Reversal ({diff_order}) for {delta} Delta Options",
        yaxis_title=f'Implied Volatility ({diff_order}) Risk Reversal',
        template='plotly_dark'
    )

    fig.show()

def single_option_delta_normalized_plot(df, option_type, delta, atm):
    fig = go.Figure()

    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"]) and option_type in column]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column and option_type in column]

    for column in filtered_cols:
        normalized_volatility = df[column] / df[atm]
        fig.add_trace(go.Scatter(
            x=df.index,
            y=normalized_volatility,
            mode='lines',
            name=column
        ))

    fig.update_layout(
        title=f"Time Series of Normalized Implied Volatility for {delta} Delta {option_type} Options",
        yaxis_title=f'Normalized Implied Volatility ({atm.upper()} ATM)',
        template='plotly_dark'
    )

    fig.show()

def normalized_delta_rr_plot(df, delta, diff_order, atm):
    fig = go.Figure()
    
    df_diff = pd.DataFrame(index=df.index)
    
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"])]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column]

    for column in filtered_cols:
        if 'Put' in column and diff_order == 'P-C':
            corresponding_call_column = column.replace('Put', 'Call')
            if corresponding_call_column in df.columns:
                difference_col_name = column.replace('Put', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_call_column]
        elif 'Call' in column and diff_order == 'C-P':
            corresponding_put_column = column.replace('Call', 'Put')
            if corresponding_put_column in df.columns:
                difference_col_name = column.replace('Call', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_put_column]
    
    df_diff = df_diff.divide(df[atm], axis=0)
    
    for column in df_diff.columns:
        fig.add_trace(go.Scatter(
            x=df_diff.index,
            y=df_diff[column],
            mode='lines',
            name=column
        ))

    fig.update_layout(
        title=f"Time Series of Normalized ({atm.upper()} Imp. Vol.) Risk Reversal ({diff_order}) for {delta} Delta Options",
        yaxis_title=f'{delta} Delta ({atm.upper()} Imp. Vol.) ({diff_order}) Risk Reversal',
        template='plotly_dark'
    )

    fig.show()

def plot_atm(df):
    fig = go.Figure()

    atm_cols = [col for col in df.columns if col.startswith("atm")]

    for column in atm_cols:
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[column],
            mode='lines',
            name=column.upper()
        ))

    fig.update_layout(
        title="Time Series of ATM Implied Volatilities",
        yaxis_title="ATM Implied Volatility",
        template='plotly_dark'
    )

    fig.show()

def normalized_delta_rr_and_single_subplot(df, delta, diff_order, atm):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    
    df_diff = pd.DataFrame(index=df.index)
    
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"])]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column]

    for column in filtered_cols:
        if 'Put' in column and diff_order == 'P-C':
            corresponding_call_column = column.replace('Put', 'Call')
            if corresponding_call_column in df.columns:
                difference_col_name = column.replace('Put', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_call_column]
        elif 'Call' in column and diff_order == 'C-P':
            corresponding_put_column = column.replace('Call', 'Put')
            if corresponding_put_column in df.columns:
                difference_col_name = column.replace('Call', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_put_column]
    
    df_diff = df_diff.divide(df[atm], axis=0)
    
    for column in df_diff.columns:
        fig.add_trace(go.Scatter(
            x=df_diff.index,
            y=df_diff[column],
            mode='lines',
            name=column
        ), row=1, col=1)
        
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df[atm],
        mode='lines',
        name=f"{delta} Delta ATM",
        line=dict(color='white')
    ), row=2, col=1)

    fig.update_layout(
        title=f"Time Series of Normalized ({atm.upper()} Imp. Vol.) Risk Reversal ({diff_order}) for {delta} Delta Options",
        yaxis_title=f'{delta} Delta ({atm.upper()} Imp. Vol.) ({diff_order}) RR',
        template='plotly_dark'
    )

    fig.show()

def normalized_delta_rr_and_multiple_subplot(df, delta, diff_order, atm):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    
    df_diff = pd.DataFrame(index=df.index)
    
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"])]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column]

    for column in filtered_cols:
        if 'Put' in column and diff_order == 'P-C':
            corresponding_call_column = column.replace('Put', 'Call')
            if corresponding_call_column in df.columns:
                difference_col_name = column.replace('Put', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_call_column]
        elif 'Call' in column and diff_order == 'C-P':
            corresponding_put_column = column.replace('Call', 'Put')
            if corresponding_put_column in df.columns:
                difference_col_name = column.replace('Call', 'Risk Reversal')
                df_diff[difference_col_name] = df[column] - df[corresponding_put_column]

    df_diff = df_diff.divide(df[atm], axis=0)
    
    for column in df_diff.columns:
        fig.add_trace(go.Scatter(
            x=df_diff.index,
            y=df_diff[column],
            mode='lines',
            name=column
        ), row=1, col=1)
        
    atm_cols = [col for col in df.columns if col.startswith("atm")]

    for column in atm_cols:
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[column],
            mode='lines',
            name=column.upper()
        ), row=2, col=1)

    fig.update_layout(
        title=f"Time Series of Normalized ({atm.upper()} Imp. Vol.) Risk Reversal ({diff_order}) for {delta} Delta Options",
        yaxis_title=f'{delta} Delta ({atm.upper()} Imp. Vol.) ({diff_order}) RR',
        template='plotly_dark'
    )

    fig.show()

def single_option_delta_normalized_plot_with_iv(df, option_type, delta, atm):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"]) and option_type in column]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column and option_type in column]

    for column in filtered_cols:
        normalized_volatility = df[column] / df[atm]
        fig.add_trace(go.Scatter(
            x=df.index,
            y=normalized_volatility,
            mode='lines',
            name=column
        ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df.index,
        y=df[atm],
        mode='lines',
        name=atm.upper(),
        line=dict(color='white')
    ), row=2, col=1)

    fig.update_layout(
        title=f"Time Series of Normalized Implied Volatility for {delta} Delta {option_type} Options with {atm.upper()}",
        yaxis_title='Normalized Implied Volatility and ATM Volatility',
        template='plotly_dark'
    )

    fig.show()

def term_structure(df, option_type, delta, timestamp):
    fig = go.Figure()
    
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"]) and option_type in column]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column and option_type in column]

    maturities = [int(column.split('Delta')[-1][:-len(option_type)]) for column in filtered_cols]
    implied_vols = df.loc[timestamp, filtered_cols]

    fig.add_trace(go.Scatter(
        x=maturities,
        y=implied_vols,
        mode='lines+markers',
        name=f'{delta} Delta {option_type} Options'
    ))

    fig.update_layout(
        title=f"Term Structure of Implied Volatility for {delta} Delta {option_type} Options on {timestamp}",
        xaxis_title='Maturity',
        yaxis_title='Implied Volatility',
        template='plotly_dark'
    )

    fig.show()

import plotly.graph_objects as go

def multi_term_structure(df, option_type, delta, timestamp):
    fig = go.Figure()

    # Define possible deltas and maturities
    deltas = ['ThirtyFive', 'TwentyFive', 'Fifteen', 'Five']
    maturities = [7, 30, 60, 90, 180]

    # Filter columns based on the input delta and option type
    if delta == "Five":
        filtered_cols = [column for column in df.columns if "Five" in column and not any(prefix in column for prefix in ["Thirty", "Twenty", "Fifteen"]) and option_type in column]
    else:
        filtered_cols = [column for column in df.columns if f"{delta}Delta" in column and option_type in column]

    # Add traces for the specified delta
    maturities_specified_delta = [int(column.split('Delta')[-1][:-len(option_type)]) for column in filtered_cols]
    implied_vols_specified_delta = df.loc[timestamp, filtered_cols]
    
    fig.add_trace(go.Scatter(
        x=maturities_specified_delta,
        y=implied_vols_specified_delta,
        mode='lines+markers',
        name=f'{delta} Delta {option_type} Option',
        line=dict(width=4)
    ))

    # Add traces for all the maturities
    for d in deltas:
        filtered_cols = [column for column in df.columns if f"{d}Delta" in column and option_type in column]
        implied_vols = df.loc[timestamp, filtered_cols]
        fig.add_trace(go.Scatter(
            x=maturities,
            y=implied_vols,
            mode='lines+markers',
            name=f'{d} Delta {option_type} Option'
        ))

    fig.update_layout(
        title=f"Term Structure of Implied Volatility on {timestamp}",
        xaxis_title='Maturity',
        yaxis_title='Implied Volatility',
        template='plotly_dark'
    )

    fig.show()

#### Delta Plot: "Implied Volatility Over Time for a Specific Option Type and Delta"

This plot shows the time series of implied volatility for a specified delta and option type (Put or Call). It is useful for observing how the implied volatility changes over time. This can inform option trading decisions and risk management, as high or increasing implied volatility might indicate more uncertainty about future price movements of the underlying asset.

In [ ]:
option_type = "Call"  # Replace with "Put" or "Call"
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
delta_plot(df_hist, option_type, delta)

#### delta_rr_plot: "Implied Volatility Risk Reversal Over Time for a Specific Delta"

This plot shows the time series of implied volatility risk reversals for a specified delta. A risk reversal is the difference in implied volatility between call and put options. The plot is useful for understanding market sentiment. If the risk reversal is positive, it indicates that call options are more expensive than put options, suggesting a bullish market sentiment. Conversely, a negative risk reversal indicates a bearish market sentiment.

In [ ]:
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
diff_order = "P-C"  # Replace with "P-C" or "C-P" (P = Put, C = Call)
delta_rr_plot(df_hist, delta, diff_order)

#### single_option_delta_normalized_plot: "Normalized Implied Volatility Over Time for a Specific Option Type and Delta"

This plot shows the time series of normalized implied volatility for a specified delta and option type, where the normalization is done by dividing by the At-The-Money (ATM) implied volatility. The plot is useful for comparing the behavior of options across different time periods, different underlying assets, or different maturities. By normalizing the implied volatility, we can remove the overall level of market volatility from our analysis, making comparisons more meaningful.

In [ ]:
option_type = "Call"  # Replace with "Put" or "Call"
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
atm = "atm30"  # Replace with "atm7", "atm30", "atm60", "atm90", or "atm180"
single_option_delta_normalized_plot(df_hist, option_type, delta, atm)

#### normalized_delta_rr_plot: "Normalized Implied Volatility Risk Reversal Over Time for a Specific Delta"

Similar to the delta_rr_plot, this plot shows the time series of implied volatility risk reversals, but with the addition of normalization by the ATM implied volatility. This plot provides a clearer view of how risk reversals change over time, independent of the overall market volatility. This can be useful for identifying changes in market sentiment over time.

In [ ]:
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
diff_order = "P-C"  # Replace with "P-C" or "C-P" (P = Put, C = Call)
atm = "atm180"  # Replace with "atm7", "atm30", "atm60", "atm90", or "atm180"
normalized_delta_rr_plot(df_hist, delta, diff_order, atm)

#### plot_atm: "At-The-Money Implied Volatility Over Time"

This plot shows the time series of ATM implied volatilities. ATM options are often the most liquid and are therefore a good indicator of the market's expectation of future volatility. This plot can be used to observe changes in market volatility over time.

In [ ]:
plot_atm(df_hist)

#### normalized_delta_rr_and_single_subplot: "Comparative Subplot of Normalized Implied Volatility and Risk Reversal Over Time"

This plot combines the normalized_delta_rr_plot and the single_option_delta_normalized_plot into a single subplot. This can be useful for comparing the behavior of risk reversals and individual options side-by-side.

In [ ]:
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
diff_order = "P-C"  # Replace with "P-C" or "C-P" (P = Put, C = Call)
atm = "atm7"  # Replace with "atm7", "atm30", "atm60", "atm90", or "atm180"
normalized_delta_rr_and_single_subplot(df_hist, delta, diff_order, atm)

#### normalized_delta_rr_and_multiple_subplot: "Comparative Subplot of Normalized Implied Volatility, Risk Reversal, and Multiple ATM Over Time"

Similar to the normalized_delta_rr_and_single_subplot plot, but with the addition of multiple ATM implied volatilities. This plot can be used to observe and compare the behavior of risk reversals and multiple ATM options side-by-side.

In [ ]:
option_type = "Put"  # Replace with "Put" or "Call"
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
atm = "atm7"  # Replace with "atm7", "atm30", "atm60", "atm90", or "atm180"
single_option_delta_normalized_plot_with_iv(df_hist, option_type, delta, atm)

In [ ]:
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
diff_order = "P-C"  # Replace with "P-C" or "C-P" (P = Put, C = Call)
atm = "atm180"  # Replace with "atm7", "atm30", "atm60", "atm90", or "atm180"
normalized_delta_rr_and_multiple_subplot(df_hist, delta, diff_order, atm)

In [ ]:
option_type = "Put"  # Replace with "Put" or "Call"
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
date_time = '2023-06-20 00:00:00'
term_structure(df_hist, option_type, delta, date_time)

In [ ]:
option_type = "Call"  # Replace with "Put" or "Call"
delta = "Five"  # Replace with the desired delta value: "Five", "Fifteen", "TwentyFive", or "ThirtyFive"
date_time = '2023-06-21 00:00:00'
multi_term_structure(df_hist, option_type, delta, date_time)

In [ ]:
# Melt the dataframe
df_melt = df.melt(id_vars=['date', 'currency', 'spot'], var_name='variable', value_name='volatility')

# Dictionary to map string deltas to integers
delta_mapping = {
    'Five': 5,
    'Fifteen': 15,
    'TwentyFive': 25,
    'ThirtyFive': 35
}

# Parse variable to get delta, expiration, and option type
df_melt['delta'] = df_melt['variable'].apply(lambda x: re.findall(r'(\w+)Delta', x)[0] if re.findall(r'(\w+)Delta', x) else None)
df_melt['delta'] = df_melt['delta'].apply(lambda x: delta_mapping.get(x, np.nan))
df_melt['expiration'] = df_melt['variable'].apply(lambda x: re.findall(r'Delta(\d+)', x)[0] if re.findall(r'Delta(\d+)', x) else None)
df_melt['option_type'] = df_melt['variable'].apply(lambda x: 'Call' if 'Call' in x else 'Put')

# Drop None rows and convert to numeric
df_melt = df_melt.dropna()
df_melt['expiration'] = pd.to_numeric(df_melt['expiration'])

# Group by delta and expiration, and average the volatility
df_melt = df_melt.groupby(['delta', 'expiration'], as_index=False).mean()

# Pivot to create a grid of delta, expiration, and volatility
pivot_table = df_melt.pivot(index='delta', columns='expiration', values='volatility')

# Interpolate missing values
pivot_table = pivot_table.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

# Convert pivot table to np array
X = pivot_table.columns.values
Y = pivot_table.index.values
Z = pivot_table.values

X, Y = np.meshgrid(X, Y)

# Create 3D surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='thermal')])

# Add labels
fig.update_layout(
                title='BTC Fixed Expiration Implied Volatiliy Delta Surface',
                scene = dict(
                    xaxis_title='Expiration',
                    yaxis_title='Delta',
                    zaxis_title='Implied Volatility'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=40))

fig.show()